In [1]:
import os
import sys

sys.path.insert(1, os.path.join(sys.path[0], ".."))

import numpy as np
import matplotlib.pyplot as plt
from argparse import ArgumentParser
import torch

import data_utils.utils as data_utils
import inference.utils as inference_utils
import BigGAN_PyTorch.utils as biggan_utils
from data_utils.datasets_common import pil_loader
import torchvision.transforms as transforms
import time
from glob import glob

import torch
import torchvision
from glob import glob
import utils
import os
from PIL import Image
from data_utils.resnet import resnet50
import torch.nn as nn
import timm
from metrics import metric_utils
import torchvision.transforms as T
import torch.nn.functional as F
from PIL import Image

Faiss library not found!


In [2]:
splitty = 1
torch.cuda.set_device(splitty)
os.environ["CUDA_VISIBLE_DEVICES"]=str(splitty)

In [3]:
def get_model(exp_name, root_path, backbone, device="cuda"):
    
    parser = biggan_utils.prepare_parser()
    parser = biggan_utils.add_sample_parser(parser)
    parser = inference_utils.add_backbone_parser(parser)

    args = ["--experiment_name", exp_name]
    args += ["--base_root", root_path]
    args += ["--model_backbone", backbone]

    config = vars(parser.parse_args(args=args))

    # Load model and overwrite configuration parameters if stored in the model
    config = biggan_utils.update_config_roots(config, change_weight_folder=False)
    generator, config = inference_utils.load_model_inference(config, device=device)
    biggan_utils.count_parameters(generator)
    generator.eval()

    return generator

In [4]:
# net = utils.load_pretrained_feature_extractor(
#         '/Work2/Watch_This/ICGAN/ic_gan/pretrained_models_path/swav_800ep_pretrain.pth.tar',
#         "selfsupervised",
#         "resnet50",
#     )
# # net = torch.nn.DataParallel(net)
# net.to("cuda")
# net.eval()
# norm_mean = torch.Tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1).to("cuda")
# norm_std = torch.Tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1).to("cuda")

In [6]:
exp_name = "%s_%s_%s_res%i%s" % (
        "icgan",
        "biggan",
        "imagenet",
        256,
        "",
    )
generator = get_model(
        exp_name, "/Work2/Watch_This/ICGAN/ic_gan/pretrained_models_path", "biggan"
    )

generator.cuda()
generator.eval()

Pegging all root folders to base root /Work2/Watch_This/ICGAN/ic_gan/pretrained_models_path
For name best  best0  we have an FID:  22.453704833984375
Checkpoint with name  best1  not in folder.
Final name selected is  best0
Loading best0 weights from /Work2/Watch_This/ICGAN/ic_gan/pretrained_models_path/icgan_biggan_imagenet_res256...
Experiment name is icgan_biggan_imagenet_res256
Adding attention layer in G at resolution 64
Param count for Gs initialized parameters: 90014147
Loading weights...
Loading best0 weights from /Work2/Watch_This/ICGAN/ic_gan/pretrained_models_path/icgan_biggan_imagenet_res256...
Putting G in eval mode..
Number of parameters: 90014340


Generator(
  (activation): ReLU()
  (shared): Embedding(1000, 128)
  (shared_feat): SNLinear(in_features=2048, out_features=512, bias=True)
  (linear): SNLinear(in_features=17, out_features=24576, bias=True)
  (blocks): ModuleList(
    (0): ModuleList(
      (0): GBlock(
        (activation): ReLU()
        (conv1): SNConv2d(1536, 1536, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): SNConv2d(1536, 1536, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv_sc): SNConv2d(1536, 1536, kernel_size=(1, 1), stride=(1, 1))
        (bn1): ccbn(
          out: 1536, in: 529, cross_replica=False
          (gain): SNLinear(in_features=529, out_features=1536, bias=False)
          (bias): SNLinear(in_features=529, out_features=1536, bias=False)
        )
        (bn2): ccbn(
          out: 1536, in: 529, cross_replica=False
          (gain): SNLinear(in_features=529, out_features=1536, bias=False)
          (bias): SNLinear(in_features=529, out_features=1536, bias=F

In [5]:
device = "cuda"

vgg16_url = 'https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/metrics/vgg16.pkl'
vgg16 = metric_utils.get_feature_detector(vgg16_url, device=device)
vgg16.cuda()
vgg16.eval()

exp_name = "%s_%s_%s_res%i%s" % (
        "icgan",
        "biggan",
        "imagenet",
        256,
        "",
    )
generator = get_model(
        exp_name, "/Work2/Watch_This/ICGAN/ic_gan/pretrained_models_path", "biggan", device=device
    )

generator.cuda()
generator.eval()

dataset = utils.get_dataset_images(
        256,
        data_path="/Work1/imagenet",
        longtail=False,
        split="train",
        test_part=False,
        which_dataset="imagenet",
        instance_json="",
        stuff_json="",
        get_encodings=True,
    )

generator1 = torch.Generator().manual_seed(42)
dataset = torch.utils.data.random_split(dataset, [0.25, 0.25, 0.25, 0.25], generator=generator1)[splitty]

loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=41,
    num_workers=12,
    pin_memory=True,
    drop_last=False,
)

num_steps = 100
initial_learning_rate = 0.1

VGG16(
  (layers): ModuleDict(
    (conv1): Conv2dLayer()
    (conv2): Conv2dLayer()
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv3): Conv2dLayer()
    (conv4): Conv2dLayer()
    (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv5): Conv2dLayer()
    (conv6): Conv2dLayer()
    (conv7): Conv2dLayer()
    (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv8): Conv2dLayer()
    (conv9): Conv2dLayer()
    (conv10): Conv2dLayer()
    (pool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv11): Conv2dLayer()
    (conv12): Conv2dLayer()
    (conv13): Conv2dLayer()
    (pool5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (fc1): FullyConnectedLayer()
    (fc2): FullyConnectedLayer()
    (fc3): Linear(in_features=4096, out_features=1000, bias=True)
    (softmax): Softmax(dim=1)
  )
)

In [ ]:
gpu = torch.device("cuda")
scaler = torch.cuda.amp.GradScaler()
for ind, data in enumerate(loader):
    # if (((ind/len(loader))*100.0) < 34.35133770822817):
    #     print("skipping")
    #     continue
        
    with torch.no_grad():
#         start_time = time.time()
        # data2 = data[0].cuda(gpu, non_blocking=True)
        # x_tf1 = data2 * 0.5 + 0.5
        # x_tf2 = (x_tf1 - norm_mean) / norm_std
        # x_tf3 = torch.nn.functional.upsample(x_tf2, 224, mode="bicubic")
        # x_feat1, _ = net(x_tf3)
        # x_feat2 =  x_feat1/torch.linalg.norm(x_feat1, dim=1, keepdims=True)

        
        path = data[3]
        data0 = data[0].cuda(gpu, non_blocking=True)
        x_feat2 =  data[4].cuda(gpu, non_blocking=True)

        z1 = torch.zeros((data0.shape[0], 119)).cuda(gpu)
        
        x_img_orig = generator(z1, None, x_feat2)

        target_img = torch.clamp((data0 * 0.5 + 0.5), 0, 1)*255
        target_features = vgg16(F.interpolate(target_img, size=(224, 224)), resize_images=False, return_lpips=True)
    
    z2 = torch.tensor(z1, dtype=torch.float32, device=gpu, requires_grad=True)
    
    optimizer = torch.optim.Adam([z2], betas=(0.9, 0.999), lr=initial_learning_rate)
    scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=0.1, total_iters=20)

    with torch.no_grad():
        original_synth_image = generator(z2, None, x_feat2).cpu()
    
    for step in range(num_steps):          
        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast():
            x_img = generator(z2, None, x_feat2)
            synth_images = (x_img + 1) * (255/2)
            synth_features = vgg16(F.interpolate(synth_images, size=(224, 224)), resize_images=False, return_lpips=True)
            lpips_loss = (target_features - synth_features).square().sum(dim=1).sum()
#             mse_loss = l2_criterion(target_img, synth_images)
            loss = lpips_loss
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        
    with torch.no_grad():    
        # x_img = generator(z2, None, x_feat2)
        for ind3 in range(x_img.shape[0]):
            folder = path[ind3].replace("imagenet", "ICGAN_Inversion").replace(".JPEG", "")
            os.makedirs(folder, exist_ok=True)
            file_name = folder.split("/")[-1]
            original_image = Image.open(path[ind3]).convert("RGB") 
            original_image.save(folder + "/" + file_name + "_R.JPEG")
            _gen_img = torchvision.transforms.functional.to_pil_image(torch.clamp((x_img[ind3] * 0.5 + 0.5), 0, 1))
            _gen_img.save(folder + "/" + file_name + "_0_G.JPEG")
            _original_synth_image = torchvision.transforms.functional.to_pil_image(torch.clamp((original_synth_image[ind3] * 0.5 + 0.5), 0, 1))
            _original_synth_image.save(folder + "/" + file_name + "_1_G.JPEG")
    #         break
    print((ind/len(loader))*100.0)
    torch.save(((ind/len(loader))*100.0), "Encoder_Inversion" + str(splitty) + "_Checkpoint.pt")

/tmp/ipykernel_60486/1051938831.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  z2 = torch.tensor(z1, dtype=torch.float32, device=gpu, requires_grad=True)
/home/sarmst/anaconda3/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


0.0
0.012800819252432157
0.025601638504864313


In [ ]:
# original_image

In [ ]:
# _original_synth_image

In [ ]:
# _gen_img

In [ ]:
# folder

In [ ]:
# file_name